In [1]:
import pandas as pd
import mysql.connector
from typing import Dict, List
from dotenv import load_dotenv # pip install python-dotenv
import os

In [2]:
# Cargar variables desde el archivo .env
load_dotenv(dotenv_path=".env")

True

In [3]:
def connect_to_database():
    """Establece la conexión con la base de datos MySQL"""
    host = os.getenv("host")
    user = os.getenv("user")
    password = os.getenv("password")
    database = os.getenv("database")

    return mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

In [4]:
def process_excel(file_path: str) -> List[tuple]:
    """
    Procesa el archivo CSV de tipos de eventos y retorna una lista de tuplas
    para inserción en la base de datos
    """
    # Leer el CSV
    df = pd.read_excel(file_path)
    
    # Limpiar los datos
    event_types = []
    for _, row in df.iterrows():
        # Verificar si tenemos un eventTypeId válido
        try:
            event_type_id = int(row['typeId'])
        except (ValueError, TypeError):
            continue
            
        # Procesar solo si tenemos un ID válido
        if pd.notna(event_type_id):
            event_types.append((
                event_type_id,
                str(row['EVENT NAME']) if pd.notna(row['EVENT NAME']) else None,
                str(row['DESCRIPTION']) if pd.notna(row['DESCRIPTION']) else None,
                str(row['ASSOCIATED qualifierId VALUES']) if pd.notna(row['ASSOCIATED qualifierId VALUES']) else None
            ))
    
    return event_types

def insert_event_types(cursor, event_types_data: List[tuple]):
    """
    Inserta o actualiza los tipos de eventos en la base de datos
    """
    upsert_query = """
    INSERT INTO eventTypes 
        (eventTypeId, eventTypeName, eventTypeDescription, eventTypeAssociatedQualifierId)
    VALUES 
        (%s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        eventTypeName = VALUES(eventTypeName),
        eventTypeDescription = VALUES(eventTypeDescription),
        eventTypeAssociatedQualifierId = VALUES(eventTypeAssociatedQualifierId)
    """
    
    try:
        cursor.executemany(upsert_query, event_types_data)
        print(f"Procesados {len(event_types_data)} tipos de eventos")
        print(f"Insertados/Actualizados: {cursor.rowcount // 2} registros")
    except Exception as e:
        print(f"Error al procesar tipos de eventos: {str(e)}")
        raise

def verify_data(cursor):
    """
    Verifica los datos insertados
    """
    query = "SELECT * FROM eventTypes ORDER BY eventTypeId"
    cursor.execute(query)
    records = cursor.fetchall()
    print(f"\nTotal de tipos de eventos en la base de datos: {len(records)}")
    print("\nPrimeros 5 registros como ejemplo:")
    for record in records[:5]:
        print(record)






    


In [5]:
def main():
    try:
        # Establecer conexión
        connection = connect_to_database()
        cursor = connection.cursor()
        
        # Procesar el CSV
        print("Procesando archivo CSV...")
        event_types_data = process_excel('Opta_typeId.xlsx')
        
        # Insertar/Actualizar datos
        print("\nInsertando datos en la base de datos...")
        insert_event_types(cursor, event_types_data)
        
        # Verificar los datos
        verify_data(cursor)
        
        # Confirmar cambios
        connection.commit()
        print("\nTodos los cambios fueron confirmados exitosamente")
        
    except Exception as error:
        print(f"Error: {error}")
        if 'connection' in locals() and connection.is_connected():
            connection.rollback()
            print("Se realizó rollback de los cambios")
        
    finally:
        if 'connection' in locals() and connection.is_connected():
            cursor.close()
            connection.close()
            print("Conexión cerrada")

In [6]:
if __name__ == "__main__":
    main()

Procesando archivo CSV...

Insertando datos en la base de datos...
Procesados 78 tipos de eventos
Insertados/Actualizados: 39 registros

Total de tipos de eventos en la base de datos: 78

Primeros 5 registros como ejemplo:
(1, 'Pass', 'The attempted delivery of the ball from one player to another player on the same team. A player can use any part of their body (permitted in the laws of the game) to execute a pass. Event categorization includes open play passes, goal kicks, corners and f', '1, 2, 3, 4, 5, 6, 15, 22, 23, 24, 25, 26, 29, 31, 55, 56, 74, 96, 97, 106, 107, 123, 124, 138, 140, 141, 152, 154, 155, 156, 157, 160, 168, 189, 195, 196, 198, 199, 210, 212, 213, 214, 218, 223, 224, 225, 233, 236, 237, 238, 240, 241, 266, 278, 279, 286, ')
(2, 'Offside Pass', 'A pass attempt where the intended receiving player is called offside. outcome: always set to 1', '1, 2, 3, 4, 5, 6, 7, 8, 23, 55, 56, 107, 123, 124, 140, 141, 152, 189, 241, 297, 298, 307, 345, 362')
(3, 'Take On', 'An attempt